In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, load_model
from keras.layers import *
from keras import backend as K
from keras import optimizers, callbacks, regularizers
import numpy as np
import pandas as pd
import cv2, h5py

In [ ]:
BATCH_SIZE = 128
INPUT_SIZE = (299,299)
print("Batch size:", BATCH_SIZE)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
train_generator = train_a_datagen.flow_from_directory('data2/train',
                                                      target_size=INPUT_SIZE,
                                                      batch_size=BATCH_SIZE)

train_aug_datagen = ImageDataGenerator(rotation_range=3,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       rescale=1./255,
                                       shear_range=0.1,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
train_aug_generator = train_b_datagen.flow_from_directory('data2/train',
                                                          target_size=INPUT_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

train_maxaug_datagen = ImageDataGenerator(rotation_range=3,
                                          width_shift_range=0.1,
                                          height_shift_range=0.1,
                                          rescale=1./255,
                                          shear_range=0.1,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')
train_maxaug_generator = train_b_datagen.flow_from_directory('train_aug',
                                                          target_size=INPUT_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'data2/validation',
    target_size=INPUT_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

**Model**

In [ ]:
from keras.applications.xception import Xception

input_tensor = Input(shape=(INPUT_SIZE[0], INPUT_SIZE[1], 3)) # input image

print("Building base model for Xception...")
xception_base = Xception(input_tensor=input_tensor, weights='imagenet', include_top=False)
features_xception = GlobalAveragePooling2D()(xception_base.output)
print("Done!")

In [ ]:
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.00))(features_xception)
#x = Dropout(0.3)(x)
#x = BatchNormalization()(x)
predictions = Dense(128, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=predictions)

In [ ]:
def set_trainable(boolean):
    global xception_base, inceptionresnet_base, inception_base
    for layer in xception_base.layers[:46]:
        layer.trainable = False
    for layer in xception_base.layers[46:]:
        layer.trainable = boolean
        
set_trainable(False) # default

**Training**

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16,
                                    write_grads=True , write_graph=True)
checkpoints = callbacks.ModelCheckpoint("inceptionresnet-{val_loss:.3f}-{val_acc:.3f}.h5",
                                              monitor='val_loss', verbose=1, save_best_only=True,
                                              save_weights_only=False, mode='auto', period=0)
reduce_on_plateau = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6, verbose=1,
                                      mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
adadelta = optimizers.Adadelta(lr=2.0, rho=0.95, epsilon=None, decay=0)
sgd = optimizers.SGD(lr=0.1, momentum=0.6, decay=0.2, nesterov=True)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
!nvidia-settings -a [gpu:0]/GPUFanControlState=1
!nvidia-settings -a [fan:0]/GPUTargetFanSpeed=90
!rm -R logs

In [ ]:
print("Training Progress:")
model_log = model.fit_generator(train_a_generator, validation_data=validation_generator,
          epochs=1, workers=5, use_multiprocessing=True,
          callbacks=[checkpoints])
model_log = model.fit_generator(train_b_generator, validation_data=validation_generator,
          epochs=1, workers=5, use_multiprocessing=True,
          callbacks=[checkpoints])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['acc'])
print("Training Progress:")
model_log = model.fit_generator(train_b_generator, validation_data=validation_generator,
          epochs=2, workers=5, use_multiprocessing=True,
          callbacks=[checkpoints])
model_log = model.fit_generator(train_a_generator, validation_data=validation_generator,
          epochs=2, workers=5, use_multiprocessing=True,
          callbacks=[checkpoints])

In [ ]:
!nvidia-settings -a [gpu:0]/GPUFanControlState=0

**Fine-tuning**

In [ ]:
BATCH_SIZE = 44

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
train_generator = train_a_datagen.flow_from_directory('data2/train',
                                                      target_size=INPUT_SIZE,
                                                      batch_size=BATCH_SIZE)

train_aug_datagen = ImageDataGenerator(rotation_range=3,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       rescale=1./255,
                                       shear_range=0.1,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
train_aug_generator = train_b_datagen.flow_from_directory('data2/train',
                                                          target_size=INPUT_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

train_maxaug_datagen = ImageDataGenerator(rotation_range=3,
                                          width_shift_range=0.1,
                                          height_shift_range=0.1,
                                          rescale=1./255,
                                          shear_range=0.1,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          fill_mode='nearest')
train_maxaug_generator = train_b_datagen.flow_from_directory('train_aug',
                                                          target_size=INPUT_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')
    
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'data2/validation',
    target_size=INPUT_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:
!nvidia-settings -a [gpu:0]/GPUFanControlState=1
!nvidia-settings -a [fan:0]/GPUTargetFanSpeed=90

In [ ]:
sgd_low = optimizers.SGD(lr=0.1, momentum=0.3, decay=0.3, nesterov=False)
options = [[True, train_a_generator, sgd_low], [True, train_b_generator, sgd_low], [True, train_a_generator, sgd_low]]
for option in options:
    set_trainable(option[0])
    model.compile(optimizer=option[2], loss='categorical_crossentropy',metrics=['acc'])
    print("Training Progress for", option,":")
    model_log = model.fit_generator(option[1], validation_data=validation_generator,
              epochs=4, workers=5, use_multiprocessing=True,
              callbacks=[checkpoints])

In [ ]:
!nvidia-settings -a [gpu:0]/GPUFanControlState=0

**Evaluation**

TODO: Fix class mapping

In [ ]:
"""from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

from glob import glob
class_names = glob("train_aug/*") # Reads all the folders in which images are present
class_names = sorted(class_names) # Sorting them
fixed_classes = []
for class_name in class_names:
    fixed_classes.append(class_name[10:])
name_id_map = dict(zip(range(len(class_names)), fixed_classes))
og_classes = [str(x) for x in range(1,129)]

In [ ]:
"""validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    'data2/validation', shuffle=False,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
"""Y_pred = model.predict_generator(validation_generator, 6322 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
"""corr_preds = []
for pred in y_pred:
    corr_preds.append(int(name_id_map[pred]))

In [ ]:
"""print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=og_classes))